In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
df = pd.read_csv(r'C:\Users\Aditya kumar Dubey\OneDrive\Apps\Documents\Desktop\Data\Titanic-Dataset.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [16]:
df.shape

(891, 12)

In [17]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],axis = 1, inplace = True)

In [18]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [19]:
#fill missing values in Age/Embarked
#oneHotEncoding sex/ 
#train model

In [20]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [21]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [22]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 2)

In [26]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S


In [36]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [30]:
trf1 = ColumnTransformer([
    ('imput', SimpleImputer(),[2]),
    ('imput2',SimpleImputer(strategy = 'most_frequent'),[6])
], remainder='passthrough')

In [31]:
trf1.fit_transform(X_train)

array([[40.0, 'C', 1, ..., 0, 0, 27.7208],
       [4.0, 'S', 3, ..., 1, 1, 16.7],
       [47.0, 'S', 3, ..., 0, 0, 9.0],
       ...,
       [71.0, 'C', 1, ..., 0, 0, 49.5042],
       [29.78590425531915, 'S', 1, ..., 0, 0, 221.7792],
       [29.78590425531915, 'S', 1, ..., 0, 0, 25.925]], dtype=object)

In [34]:
trf1.named_transformers_['imput2'].statistics_

array(['S'], dtype=object)

In [40]:
trf2 = ColumnTransformer([
    ('ohe',OneHotEncoder(sparse_output = False,handle_unknown= 'ignore'),[1,6])
])

In [41]:
from xgboost import XGBClassifier

In [42]:
clf = XGBClassifier()

In [43]:
from sklearn.pipeline import Pipeline

In [45]:
pipe = Pipeline([
    ('step1',trf1),
    ('step2',trf2),
    ('step3',clf)
])

In [47]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imput', SimpleImputer(),
                                                  [2]),
                                                 ('imput2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('step2',
                 ColumnTransformer(transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('step3',
                 XGBClassifier(base_score=None, booster=None, callbacks=No...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [48]:
y_pred = pipe.predict(X_test)

In [49]:
from sklearn.metrics import accuracy_score

In [50]:
accuracy_score(y_test,y_pred)

0.6368715083798883

In [51]:
import pickle

In [54]:
pickle.dump(pipe,open('pipe.pkl','wb'))

In [58]:
pp = pickle.load(open('pipe.pkl','rb'))

In [60]:
pp.predict(np.array([[3,'female', 4.0,1,1,16.7000,'S']]))

C:\Users\Aditya kumar Dubey\anaconda3\envs\dubey\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
C:\Users\Aditya kumar Dubey\anaconda3\envs\dubey\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])

In [63]:
pp.predict(np.array([[1,'female',38.0,1,0,71.2833,'C']]))

C:\Users\Aditya kumar Dubey\anaconda3\envs\dubey\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
C:\Users\Aditya kumar Dubey\anaconda3\envs\dubey\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([1])